In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams['figure.figsize'] = [15, 5]

In [ ]:
df = (pd.read_csv("../data/preprocessed/Building_4.csv")
      .astype({'datetime': 'datetime64'})
      .set_index('datetime'))

# Testing quality of prediction simulation

In [ ]:
#sns.pairplot(difs)
pdf = pd.concat([
    df["outdoor_temp"],
    df["pred_6h_outdoor_temp"].shift(6),
    df["pred_12h_outdoor_temp"].shift(12),
    df["pred_24h_outdoor_temp"].shift(24),
], axis=1).iloc[24:, :]

In [ ]:
pred_temp_cols = [
    'pred_6h_outdoor_temp',
    'pred_12h_outdoor_temp',
    'pred_24h_outdoor_temp'
]

X = pdf[["pred_12h_outdoor_temp", "pred_24h_outdoor_temp"]]
y = pdf.outdoor_temp
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
pdf_test = pdf.loc[X_test.index, :].assign(
    pred_12h_better_outdoor_temp = lr.predict(X_test),
    pred_error = lambda x: x.pred_6h_outdoor_temp - x.outdoor_temp,
    pred_better_error = lambda x: x.pred_12h_better_outdoor_temp - x.outdoor_temp,
)

In [ ]:
pdf.iloc[2210:2260, :].plot()

In [ ]:
pdf_test[["pred_error", "pred_better_error"]].rename(columns={
    "pred_error": "error of 6 hour prediction from csv",
    "pred_better_error": "error of my 12 hour prediction",
    }).boxplot()